### Q1: Running Elastic

In [3]:
build_hash ="42f05b9372a9a4a470db3b52817899b99a76ee73"

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2 Indexing the data

In [2]:
from elasticsearch import Elasticsearch
Answer= 'Index'

### Q3: Searching

In [3]:
es_client = Elasticsearch('http://localhost:9200') 

In [6]:

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course-questions'}

In [36]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
from tqdm.auto import tqdm 
for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

/opt/conda/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:27<00:00, 34.52it/s]


In [8]:
query = "How do I execute a command in a running docker container?"

In [24]:
search_query={
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [43]:
response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][0]['_score']

75.54128

### Q4: Filtering

In [9]:
search_query={
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [10]:
response_2 = es_client.search(index=index_name, body=search_query)
response_2['hits']['hits'][2]

{'_index': 'course-questions',
 '_id': 'DsttfZABSgNCbHCCkpck',
 '_score': 49.938507,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

### Q5: Building a prompt

In [11]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [12]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [13]:
context_pieces=[]
for hit in response_2['hits']['hits']:
        doc=hit['_source']
        context_piece= context_template.format(**doc)
        context_pieces.append(context_piece)
context = '\n\n'.join(context_pieces)
           

In [14]:
prompt = prompt_template.format(question=query, context=context).strip()
len(prompt)

2714

### Q6. Tokens

In [22]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt))
encoding.decode_single_token_bytes(63842)

b"You're"

### Bonus

In [30]:
def llm(prompt,client):
    answer= client.chat.completions.create(model='gpt-3.5-turbo-0125',
                                           messages=[{"role": "user","content":prompt}])
    return answer.choices[0].message.content

In [32]:
from openai import OpenAI
client = OpenAI()
llm(prompt,client)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable